## 05卷积神经网络
___

In [11]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [12]:
# 配置设备
device = torch.device('cpu')

In [13]:
# 设置超参数
num_epochs = 5
num_classes = 10 
batch_size = 100
learning_rate = 0.001
# MNIST数据集
train_dataset = torchvision.datasets.MNIST(root='/home/wangye/data',
                                          train=True,
                                          transform=transforms.ToTensor(),
                                          download=True)
test_dataset = torchvision.datasets.MNIST(root='/home/wangye/data',
                                         train=False,
                                         transform=transforms.ToTensor())
# 数据加载
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

In [14]:
# 定义包含两个卷积层的卷积神经网络
class ConvNet(nn.Module):
    def __init__(self,num_classes=10):
        super(ConvNet,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,16,kernel_size=5,stride=1,padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size=5,stride=1,padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.fc = nn.Linear(7*7*32,num_classes)
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
#         print(out.shape)
        out = out.reshape(out.size(0),-1)
        out = self.fc(out)
        return out

## 上述代码讲解：
1. class torch.nn.Sequential(* args)
    * 这是一个时序的容器，可以按照我们传入的顺序添加进去，帮助我们快速搭建神经网络
2. class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True):
    * 二维卷积层，输入尺寸为（N,C_in,H_in,W_in）输出尺寸为(N,C_out,H_out,W_out)
    * 参数讲解：
        * in_channels:输入的通道数,比如说灰色图片通道为1，彩色为3（RGB）
        * out_channels:输出的通道数，输出的通道数跟卷积核的数量有关，或者说等于卷积核的数量。
        * kernel_size:卷积核大小
        * stride:表示步长
        * padding:为了保持输入和输出相同，补0的层数
            * 根据公式$1+2*2-5+1=1$可见padding设置为2刚好满足要求
3. class torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True)：
    * 对输入的数据进行标准化处理，现在不需要深究，以后会展开讲
    * 参数讲解：
        * num_features:期望输入的特征数
        * 其他参数默认设置就好，不作讲解
4. class torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)：
    * 对输入的数据做二维的最大池化操作
    * 参数讲解：
         * kernel_size: max pooling的窗口大小
         * stride:窗口移动的步长
         * 其余参数可以参考文档，此处不展开讲解
         
___
         

## 神经网络的输入输出shape解释：
其实非常简单，只要熟悉卷积核运算后的输出shape1公式，和max池化后的输出shape2公式，问题便迎刃而解
1. shape_1公式：
    $W_{out} = \frac{W_{in}+2*P-F}{S}+1$
2. shape_2公式：
    $W_{out} =\frac{W_{in}-F}{S}+1$
3. 参数：
    * W即为输入的宽，长也一样
    * P 为Padding的长度
    * F 为滤波器的大小
    * S 为步长
对于本网络的具体推导，以图片的形式展示出来：<br>

![shape推导](https://github.com/wangye8899/Pytorch/blob/master/IMAGES/gaitubao_13254023_-90.jpg?raw=true)
___

In [15]:
# 实例化网络
model = ConvNet(num_classes).to(device)

In [16]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [17]:
# 训练模型
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i ,(images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
#         前向传播
        outputs = model(images)
        loss = criterion(outputs,labels)
#         反向传播，更新参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1)%100 == 0:
            print('Epoch[{}/{}],Step[{}/{}],Loss:{:.4f}'.format(epoch+1,num_epochs,i+1,
                                                               total_step,
                                                               loss.item()))

Epoch[1/5],Step[100/600],Loss:0.2065
Epoch[1/5],Step[200/600],Loss:0.0436
Epoch[1/5],Step[300/600],Loss:0.1447
Epoch[1/5],Step[400/600],Loss:0.0394
Epoch[1/5],Step[500/600],Loss:0.0531
Epoch[1/5],Step[600/600],Loss:0.0421
Epoch[2/5],Step[100/600],Loss:0.0518
Epoch[2/5],Step[200/600],Loss:0.0249
Epoch[2/5],Step[300/600],Loss:0.0132
Epoch[2/5],Step[400/600],Loss:0.0658
Epoch[2/5],Step[500/600],Loss:0.0299
Epoch[2/5],Step[600/600],Loss:0.0705
Epoch[3/5],Step[100/600],Loss:0.0104
Epoch[3/5],Step[200/600],Loss:0.0177
Epoch[3/5],Step[300/600],Loss:0.0112
Epoch[3/5],Step[400/600],Loss:0.0203
Epoch[3/5],Step[500/600],Loss:0.0404
Epoch[3/5],Step[600/600],Loss:0.0737
Epoch[4/5],Step[100/600],Loss:0.0439
Epoch[4/5],Step[200/600],Loss:0.0209
Epoch[4/5],Step[300/600],Loss:0.0264
Epoch[4/5],Step[400/600],Loss:0.0179
Epoch[4/5],Step[500/600],Loss:0.0199
Epoch[4/5],Step[600/600],Loss:0.0138
Epoch[5/5],Step[100/600],Loss:0.0346
Epoch[5/5],Step[200/600],Loss:0.0044
Epoch[5/5],Step[300/600],Loss:0.0199
E

In [23]:
# 测试模型
model.eval()
with torch.no_grad():
    correct = 0
    total  = 0
    for images , labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()
    print('10000条测试数据的准确率为：{}'.format(100*correct/total))
torch.save(model.state_dict(),'model.ckpt')
print(total)
print(correct)

10000条测试数据的准确率为：98.9
10000
9890
